In [80]:
import os
import json
import IPython
from scipy.io import wavfile

In [88]:
def get_transcription(path2transcription):
    transcription = []
    offset = -1
    end = 0
    with open(path2transcription, 'rt') as f:
        records = f.readlines()
        for sentence in records:
            fields = sentence.split()
            label = fields[6:]
            if not 'ignore_time_segment_in_scoring' in label:
                if offset < 0:
                    offset = float(fields[3])
                transcription.extend(label)
                end = float(fields[4])
    transcription = " ".join(transcription)
    duration = end - offset
    return offset, duration, transcription

def get_offset(path2transcription):
    
    with open(path2transcription, 'rt') as f:
        records = f.readlines()
        for sentence in records:
            fields = sentence.split()
            label = fields[6:]
            if not 'ignore_time_segment_in_scoring' in label:
                return float(fields[3])
    return 

In [48]:
offset, duration, transcription = get_transcription('/aimlx/Datasets/TEDLIUM_release1/dev/stm/BlaiseAguerayArcas_2007.stm')

In [49]:
offset

12.2

In [19]:
duration

1156.81

In [21]:
pwd

'/aimlx/kws/kws_db_creation'

In [75]:
path = '/aimlx/Datasets/TEDLIUM_release1/dev/json/BlaiseAguerayArcas_2007.json'  
def analyze_jsons():
    with open('/aimlx/Datasets/TEDLIUM_release1/dev/json/BlaiseAguerayArcas_2007.json') as json_file:
        data = json.load(json_file)
        nb_words = len(data['words'])
        nb_aligned = 0
        unaligned_words = []
        for word in data['words']:
            if word['case'] == 'success':
                if word['word'] == 'computationally':
                    print(word['start'], word['end'])
                nb_aligned += 1
            if word['case'] != 'success':
                unaligned_words.append(word['word'])
        print('Percentage of aligned words', 100*(nb_aligned/nb_words))
        print('Unaligned words : ', unaligned_words)

In [76]:
analyze_jsons()

225.09 225.85
Percentage of aligned words 94.885598923284
Unaligned words :  ['what', 'i', "'m", 'going', "'s", 'an', 'kind', "'t", 'it', "'s", 'in', 'the', 'doesn', 'you', 'that', 'it', "'s", 'that', 'it', "'s", "'t", "'s", 'you', 'want', "'t", 'now', 'a', 'graduate', 'student', 'at', 'kind', 'the', 'we', 'they', "'ve", 'and', 'know', 'it', "'s", 'very', 'it', "'s", 'to', 'this', 'kind', 'is', 'this', 'is', 'in', 'many', 'of', 'them', 'are', 'occluded', 'by', 'faces', 'and', 'and', 'so', 'of', 'of', 'of', 'do', 'i', 'do', 'i', 'understand', 'what', 'that', 'and', 'it', 'and', 'and', 'that', 'so', 'much']


In [77]:
fs, signal = wavfile.read(path.replace('json', 'wav'))
start = int(fs * (225.09 + offset))
end = int(fs * (225.85 + offset))
microsoft = signal[start:end]

In [78]:
IPython.display.Audio(microsoft, rate=fs)

In [50]:
offset

12.2

In [52]:
start

115372.2

In [97]:
def build_keyword_database(keywords, path2jsons, path2kw_db):
    for 


def extract_kw(keyword, path2jsons, path2kw_db):
    
    if not os.path.exists(os.path.join(path2kw_db, keyword)):
        os.mkdir(os.path.join(path2kw_db, keyword))
        
    for path in path2jsons:
        with open(path) as json_file:
            data = json.load(json_file)
            count = 0
            for word in data['words']:
                if word['word'] == keyword and word['case'] == 'success':
                    start, end = word['start'], word['end']
                    offset = get_offset(path2transcription=path.replace('json', 'stm'))
                    fs, signal = wavfile.read(path.replace('json', 'wav'))
                    start, end = int(fs * (start + offset)), int(fs * (end + offset))
                    filename = path.split('/')[-1].split('.')[0] + '_' + str(count) + '.wav'
                    wavfile.write(os.path.join(path2kw_db, keyword, filename), data=signal[start:end], rate=fs)
                    count += 1 
                    
def get_jsons(path='/aimlx/Datasets/TEDLIUM_release1/dev/json'):
    
    path2jsons = []
    for _, _, files in os.walk(path):
        for file in files:
            if file.endswith('.json') and 'checkpoint' not in file:
                path2jsons.append(os.path.join(path, file))    
                
    return path2jsons

In [98]:
path2jsons = get_jsons(path='/aimlx/Datasets/TEDLIUM_release1/dev/json')

In [99]:
extract_kw(keyword='people', path2jsons=path2jsons, path2kw_db='/aimlx/Datasets/TEDLIUM_release1/kw_db')